In [1]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'prod'

In [2]:
import src.config
from deltalake import DeltaTable
from pathlib import Path
import pandas as pd


In [3]:
TABLE_PATH = src.config.BRONZE / "metadata_lookup_engine1"
metadata = DeltaTable(TABLE_PATH).to_pandas()

In [4]:
metadata

,owner,table_name,column_name,data_type,load_timestamp_utc
0,SYS,DUAL,DUMMY,VARCHAR2,2025-10-08 13:45:19.387220+00:00
1,SYS,SYSTEM_PRIVILEGE_MAP,PRIVILEGE,NUMBER,2025-10-08 13:45:19.387220+00:00
2,SYS,SYSTEM_PRIVILEGE_MAP,NAME,VARCHAR2,2025-10-08 13:45:19.387220+00:00
3,SYS,SYSTEM_PRIVILEGE_MAP,PROPERTY,NUMBER,2025-10-08 13:45:19.387220+00:00
4,SYS,TABLE_PRIVILEGE_MAP,PRIVILEGE,NUMBER,2025-10-08 13:45:19.387220+00:00
...,...,...,...,...,...
46564,OSIEXTN,PS_BAMEXT_RPT,CURRACCTSTATCD,VARCHAR2,2025-10-08 13:45:19.387220+00:00
46565,OSIEXTN,PS_BAMEXT_RPT,CONTRACTDATE,DATE,2025-10-08 13:45:19.387220+00:00
46566,OSIEXTN,PS_BAMEXT_RPT,BRANCHORGNBR,NUMBER,2025-10-08 13:45:19.387220+00:00
46567,OSIEXTN,PS_BAMEXT_RPT,MJACCTTYPCD,VARCHAR2,2025-10-08 13:45:19.387220+00:00


In [5]:
# TABLE_PATH = src.config.BRONZE / "metadata_lookup_engine1"
TABLE_PATH = src.config.SILVER / "account"


In [6]:
account = DeltaTable(TABLE_PATH).to_pandas()

In [7]:
account

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,datelastmaint,Category,inactivedate,branchname,primaryownercity,primaryownerstate,primaryownerzipcd,customer_id,Macro Account Type,load_timestamp_utc
0,2025-10-07,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.0000,...,2020-02-26 23:28:03,None,NaT,BCSB - MAIN OFFICE,MINNEAPOLIS,MN,55480,O663,Other,2025-10-08 13:57:59.962357+00:00
1,2025-10-07,150586025,GREATER FALL RIVER V.S.D,Municipal Now,0.0,VAR,CK,CK27,ACT,0.0003,...,2021-03-25 22:19:28,None,NaT,BCSB - MUNI FALL RIVER BRANCH,FALL RIVER,MA,02723,O1004153,Deposit,2025-10-08 13:57:59.962357+00:00
2,2025-10-07,150887241,CITY OF TAUNTON,ICS Shadow - Muni - MMDA/SAV,0.0,None,CK,CK42,ACT,0.0000,...,NaT,None,NaT,BCSB - MUNI MAIN OFFICE,TAUNTON,MA,02780,O1002543,Deposit,2025-10-08 13:57:59.962357+00:00
3,2025-10-07,60801557,"COMMUNITY COUNSELING OF BRISTOL COUNTY, INCORP...",Investment Business Checking,0.0,VAR,CK,CK28,ACT,0.0220,...,2024-02-14 11:54:44,None,NaT,BCSB - MAIN OFFICE,TAUNTON,MA,02780,O1000047,Deposit,2025-10-08 13:57:59.962357+00:00
4,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,2024-06-11 12:02:04,CRE,NaT,BCSB - COMM'L LENDING- TAUNTON,WAKEFIELD,MA,01880,O1016877,Loan,2025-10-08 13:57:59.962357+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89515,2025-10-07,27047636,BRISTOL COUNTY SAVINGS BANK,Payoff Overages,0.0,None,BKCK,BTC2,ACT,0.0000,...,2022-09-27 22:46:27,None,NaT,BCSB - MAIN OFFICE,TAUNTON,MA,02780,O1,Other,2025-10-08 13:57:59.962357+00:00
89516,2025-10-07,151223759,"BEAUCHAMP, LAURA L.",Access Checking,100.0,None,CK,CK14,ACT,0.0000,...,2025-09-12 20:51:36,None,NaT,BCSB - PAWTUCKET BRANCH,PAWTUCKET,RI,02860,P1176441,Deposit,2025-10-08 13:57:59.962357+00:00
89517,2025-10-07,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.0000,...,2022-09-27 22:46:27,None,NaT,BCSB - MAIN OFFICE,TAUNTON,MA,02780,O1,Other,2025-10-08 13:57:59.962357+00:00
89518,2025-10-07,27020568,BRISTOL COUNTY SAVINGS BANK,PDO Checks - OLD,0.0,None,BKCK,BTC1,ACT,0.0000,...,2022-09-27 22:46:27,None,NaT,BCSB - MAIN OFFICE,TAUNTON,MA,02780,O1,Other,2025-10-08 13:57:59.962357+00:00


In [8]:
account_cleaned = account[
    (account['mjaccttypcd'].isin(['CML','MTG','MLN']))
].copy()

In [9]:
account_cleaned

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,datelastmaint,Category,inactivedate,branchname,primaryownercity,primaryownerstate,primaryownerzipcd,customer_id,Macro Account Type,load_timestamp_utc
4,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,2024-06-11 12:02:04,CRE,NaT,BCSB - COMM'L LENDING- TAUNTON,WAKEFIELD,MA,01880,O1016877,Loan,2025-10-08 13:57:59.962357+00:00
6,2025-10-07,150862011,"R3 PROJECT COMPANY, LLC",CML Fixed Construction,20000000.0,FIX,CML,CM07,ACT,0.065000,...,2023-03-13 22:23:21,CRE,2026-03-07,BCSB - COMM'L LENDING- TAUNTON,QUINCY,MA,02169,O1015409,Loan,2025-10-08 13:57:59.962357+00:00
7,2025-10-07,151038843,"POWER 250, LLC",CML ARM Construction,27500000.0,VAR,CML,CM08,ACT,0.069068,...,2024-04-24 15:47:27,CRE,2026-10-22,BCSB - COMM'L LENDING - WARWICK,EAST WALPOLE,MA,02032,O1012385,Loan,2025-10-08 13:57:59.962357+00:00
8,2025-10-07,150809211,29 CENTER STREET LLC,Commercial Mortgages,18000000.0,FIX,CML,CM40,ACT,0.049000,...,2022-11-03 14:12:28,CRE,NaT,BCSB - COMM'L LENDING- TAUNTON,TAUNTON,MA,02780,O1012657,Loan,2025-10-08 13:57:59.962357+00:00
9,2025-10-07,150649922,"JMC/STM/TGCI UNION SQUARE I, LLC",Commercial Mortgages,17250000.0,VAR,CML,CM40,ACT,0.063843,...,2021-09-13 13:47:57,CRE,2024-08-01,BCSB - COMM'L LENDING- TAUNTON,TAUNTON,MA,02780,O1012096,Loan,2025-10-08 13:57:59.962357+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89131,2025-10-07,970149281,"HUDSON, VICTORIA A.",Equity Line of Credit,25000.0,VAR,MTG,MG60,ACT,0.072500,...,2020-02-26 23:27:59,Residential,2023-02-21,BCSB - CONS INST LENDING - ATTLEBORO,SOUTH ATTEBORO,MA,02703,P1028112,Loan,2025-10-08 13:57:59.962357+00:00
89132,2025-10-07,970158171,"NOISEUX, KENNETH E.",Special HELOC,100000.0,VAR,MTG,MG52,ACT,0.072500,...,2020-02-26 23:27:59,Residential,2023-12-13,BCSB - CONS INST LENDING - PAWTUCKET,PAWTUCKET,RI,02860,P1032653,Loan,2025-10-08 13:57:59.962357+00:00
89133,2025-10-07,970170501,"MOURA, JOAO B.",Special HELOC,20000.0,VAR,MTG,MG52,ACT,0.072500,...,2020-02-26 23:27:59,Residential,2024-11-17,BCSB - CONS INST LENDING- TAUNTON,TAUNTON,MA,02780,P1032719,Loan,2025-10-08 13:57:59.962357+00:00
89134,2025-10-07,970186001,"SHURTLEFF, GIDEON N.",Special HELOC,38925.0,VAR,MTG,MG52,ACT,0.072500,...,2020-02-26 23:27:59,Residential,2025-11-16,BCSB - CONS INST LENDING - DARTMOUTH,LAKEVILLE,MA,02347,P1032808,Loan,2025-10-08 13:57:59.962357+00:00


In [10]:
customer_address_link = DeltaTable(src.config.SILVER / "customer_address_link").to_pandas()

In [11]:
customer_address_link

,customer_id,addrnbr,addrusecd,effdate,load_timestamp_utc
0,P1176121,1440865,PRI,2025-10-07,2025-10-08 14:02:42.451361+00:00
1,P1174663,1443591,HEML,2025-10-07,2025-10-08 14:02:42.451361+00:00
2,P1177202,1443569,HEML,2025-10-07,2025-10-08 14:02:42.451361+00:00
3,P1177211,1443587,PRI,2025-10-07,2025-10-08 14:02:42.451361+00:00
4,P1177212,1443588,PRI,2025-10-07,2025-10-08 14:02:42.451361+00:00
...,...,...,...,...,...
367123,O965,1408,PRI,2025-10-07,2025-10-08 14:02:42.451361+00:00
367124,O15,193,PRI,2025-10-07,2025-10-08 14:02:42.451361+00:00
367125,O142,33,PRI,2025-10-07,2025-10-08 14:02:42.451361+00:00
367126,O143,32,PRI,2025-10-07,2025-10-08 14:02:42.451361+00:00


In [12]:
customer_address_link = customer_address_link[customer_address_link['addrusecd'] == 'PRI'].copy()


In [13]:
customer_address_link = customer_address_link[[
    'customer_id',
    'addrnbr'
]].copy()

In [14]:
account_cleaned = account_cleaned.merge(customer_address_link, on='customer_id', how='left')

In [15]:
TABLE_PATH = src.config.SILVER / "address"
address = DeltaTable(TABLE_PATH).to_pandas()

In [16]:
address = address.drop(columns='load_timestamp_utc').copy()

In [17]:
import cdutils.input_cleansing

In [18]:
account_cleaned_schema = {
    'addrnbr':'str'
}
account_cleaned = cdutils.input_cleansing.cast_columns(account_cleaned, account_cleaned_schema)

In [19]:
customer_address_link_schema = {
    'addrnbr':'str'
}
customer_address_link = cdutils.input_cleansing.cast_columns(customer_address_link, customer_address_link_schema)

In [20]:
account_cleaned = account_cleaned.merge(address, on='addrnbr', how='left')

In [21]:
account_cleaned

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,primaryownerstate,primaryownerzipcd,customer_id,Macro Account Type,load_timestamp_utc,addrnbr,Full_Street_Address,cityname,statecd,zipcd
0,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,MA,01880,O1016877,Loan,2025-10-08 13:57:59.962357+00:00,1409242,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880
1,2025-10-07,150862011,"R3 PROJECT COMPANY, LLC",CML Fixed Construction,20000000.0,FIX,CML,CM07,ACT,0.065000,...,MA,02169,O1015409,Loan,2025-10-08 13:57:59.962357+00:00,1378602,1512 HANCOCK,QUINCY,MA,02169
2,2025-10-07,151038843,"POWER 250, LLC",CML ARM Construction,27500000.0,VAR,CML,CM08,ACT,0.069068,...,MA,02032,O1012385,Loan,2025-10-08 13:57:59.962357+00:00,1319663,108 ENDEAN DRIVE,EAST WALPOLE,MA,02032
3,2025-10-07,150809211,29 CENTER STREET LLC,Commercial Mortgages,18000000.0,FIX,CML,CM40,ACT,0.049000,...,MA,02780,O1012657,Loan,2025-10-08 13:57:59.962357+00:00,1369560,29 BROADWAY,TAUNTON,MA,02780
4,2025-10-07,150649922,"JMC/STM/TGCI UNION SQUARE I, LLC",Commercial Mortgages,17250000.0,VAR,CML,CM40,ACT,0.063843,...,MA,02780,O1012096,Loan,2025-10-08 13:57:59.962357+00:00,1311476,29 BROADWAY,TAUNTON,MA,02780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7912,2025-10-07,970149281,"HUDSON, VICTORIA A.",Equity Line of Credit,25000.0,VAR,MTG,MG60,ACT,0.072500,...,MA,02703,P1028112,Loan,2025-10-08 13:57:59.962357+00:00,1036987,65 ZACHARY LN,SOUTH ATTEBORO,MA,02703
7913,2025-10-07,970158171,"NOISEUX, KENNETH E.",Special HELOC,100000.0,VAR,MTG,MG52,ACT,0.072500,...,RI,02860,P1032653,Loan,2025-10-08 13:57:59.962357+00:00,1057479,7 DENVER ST,PAWTUCKET,RI,02860
7914,2025-10-07,970170501,"MOURA, JOAO B.",Special HELOC,20000.0,VAR,MTG,MG52,ACT,0.072500,...,MA,02780,P1032719,Loan,2025-10-08 13:57:59.962357+00:00,1046742,149 POWER ST,TAUNTON,MA,02780
7915,2025-10-07,970186001,"SHURTLEFF, GIDEON N.",Special HELOC,38925.0,VAR,MTG,MG52,ACT,0.072500,...,MA,02347,P1032808,Loan,2025-10-08 13:57:59.962357+00:00,1060753,3 EDNA CIR,LAKEVILLE,MA,02347


In [22]:
account_cleaned

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,primaryownerstate,primaryownerzipcd,customer_id,Macro Account Type,load_timestamp_utc,addrnbr,Full_Street_Address,cityname,statecd,zipcd
0,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,MA,01880,O1016877,Loan,2025-10-08 13:57:59.962357+00:00,1409242,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880
1,2025-10-07,150862011,"R3 PROJECT COMPANY, LLC",CML Fixed Construction,20000000.0,FIX,CML,CM07,ACT,0.065000,...,MA,02169,O1015409,Loan,2025-10-08 13:57:59.962357+00:00,1378602,1512 HANCOCK,QUINCY,MA,02169
2,2025-10-07,151038843,"POWER 250, LLC",CML ARM Construction,27500000.0,VAR,CML,CM08,ACT,0.069068,...,MA,02032,O1012385,Loan,2025-10-08 13:57:59.962357+00:00,1319663,108 ENDEAN DRIVE,EAST WALPOLE,MA,02032
3,2025-10-07,150809211,29 CENTER STREET LLC,Commercial Mortgages,18000000.0,FIX,CML,CM40,ACT,0.049000,...,MA,02780,O1012657,Loan,2025-10-08 13:57:59.962357+00:00,1369560,29 BROADWAY,TAUNTON,MA,02780
4,2025-10-07,150649922,"JMC/STM/TGCI UNION SQUARE I, LLC",Commercial Mortgages,17250000.0,VAR,CML,CM40,ACT,0.063843,...,MA,02780,O1012096,Loan,2025-10-08 13:57:59.962357+00:00,1311476,29 BROADWAY,TAUNTON,MA,02780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7912,2025-10-07,970149281,"HUDSON, VICTORIA A.",Equity Line of Credit,25000.0,VAR,MTG,MG60,ACT,0.072500,...,MA,02703,P1028112,Loan,2025-10-08 13:57:59.962357+00:00,1036987,65 ZACHARY LN,SOUTH ATTEBORO,MA,02703
7913,2025-10-07,970158171,"NOISEUX, KENNETH E.",Special HELOC,100000.0,VAR,MTG,MG52,ACT,0.072500,...,RI,02860,P1032653,Loan,2025-10-08 13:57:59.962357+00:00,1057479,7 DENVER ST,PAWTUCKET,RI,02860
7914,2025-10-07,970170501,"MOURA, JOAO B.",Special HELOC,20000.0,VAR,MTG,MG52,ACT,0.072500,...,MA,02780,P1032719,Loan,2025-10-08 13:57:59.962357+00:00,1046742,149 POWER ST,TAUNTON,MA,02780
7915,2025-10-07,970186001,"SHURTLEFF, GIDEON N.",Special HELOC,38925.0,VAR,MTG,MG52,ACT,0.072500,...,MA,02347,P1032808,Loan,2025-10-08 13:57:59.962357+00:00,1060753,3 EDNA CIR,LAKEVILLE,MA,02347


In [23]:
TABLE_PATH = src.config.SILVER / "property"
property = DeltaTable(TABLE_PATH).to_pandas()

In [24]:
property

,propnbr,aprsvalueamt,aprsdate,taxtowncd,taxtown,taxtypecd,taxdisbperiod,taxescrowyn,propaddr1,propaddr2,...,occupancyeffdate,parcelnbr,fixedmarginamt,priorityseqnbr,datereleased,calcmarginpct,rundate,nbrofunits,datelastmaint,load_timestamp_utc
0,1,None,NaT,None,None,None,None,None,None,None,...,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:26:57,2025-10-08 13:58:52.562368+00:00
1,51755,None,NaT,None,None,None,None,None,None,None,...,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:08,2025-10-08 13:58:52.562368+00:00
2,53695,None,NaT,None,None,None,None,None,None,None,...,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:08,2025-10-08 13:58:52.562368+00:00
3,66706,None,NaT,None,None,None,None,None,None,None,...,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:11,2025-10-08 13:58:52.562368+00:00
4,54774,None,NaT,None,None,None,None,None,None,None,...,NaT,None,None,NaN,NaT,1.000000,2025-10-07,0.0,2025-10-07 21:27:08,2025-10-08 13:58:52.562368+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86315,37885,None,NaT,None,None,None,None,None,None,None,...,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:05,2025-10-08 13:58:52.562368+00:00
86316,37886,890000.00,2017-06-05,None,None,None,None,None,"373 WESTMINSTER STREET, UNITS 1 & 2",None,...,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:05,2025-10-08 13:58:52.562368+00:00
86317,37887,None,NaT,None,None,None,None,None,None,None,...,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:05,2025-10-08 13:58:52.562368+00:00
86318,37888,890000.00,2017-06-05,None,None,None,None,None,"373 WESTMINSTER STREET, UNITS 1 & 2",None,...,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:05,2025-10-08 13:58:52.562368+00:00


In [25]:
property = property.drop(columns='load_timestamp_utc').copy()

In [26]:
# Acct prop linking
TABLE_PATH = src.config.SILVER / "account_property_link"
acct_prop_link = DeltaTable(TABLE_PATH).to_pandas()

In [27]:
acct_prop_link

,acctnbr,propnbr,load_timestamp_utc
0,5341,1,2025-10-08 13:58:52.577994+00:00
1,5847,51755,2025-10-08 13:58:52.577994+00:00
2,45978,53695,2025-10-08 13:58:52.577994+00:00
3,71033,66706,2025-10-08 13:58:52.577994+00:00
4,100004,54774,2025-10-08 13:58:52.577994+00:00
...,...,...,...
86490,600283181885,37885,2025-10-08 13:58:52.577994+00:00
86491,4002786570099,37886,2025-10-08 13:58:52.577994+00:00
86492,4002786570099,37887,2025-10-08 13:58:52.577994+00:00
86493,4002786570103,37888,2025-10-08 13:58:52.577994+00:00


In [28]:
acct_prop_link = acct_prop_link.drop(columns='load_timestamp_utc').copy()

In [29]:
account_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7917 entries, 0 to 7916
Data columns (total 60 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   effdate                 7917 non-null   datetime64[us]     
 1   acctnbr                 7917 non-null   object             
 2   ownersortname           7917 non-null   object             
 3   product                 7917 non-null   object             
 4   noteopenamt             7917 non-null   float64            
 5   ratetypcd               7917 non-null   object             
 6   mjaccttypcd             7917 non-null   object             
 7   currmiaccttypcd         7917 non-null   object             
 8   curracctstatcd          7917 non-null   object             
 9   noteintrate             7917 non-null   float64            
 10  bookbalance             7917 non-null   float64            
 11  notebal                 7917 non-null   flo

In [30]:
acct_prop_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86495 entries, 0 to 86494
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   acctnbr  86495 non-null  object
 1   propnbr  86495 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [31]:
merged_df = account_cleaned.merge(acct_prop_link, how='inner', on='acctnbr').merge(property, how='left', on='propnbr')

In [32]:
merged_df

,effdate_x,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,propyearnbr,occupancyeffdate,parcelnbr,fixedmarginamt,priorityseqnbr,datereleased,calcmarginpct,rundate,nbrofunits,datelastmaint_y
0,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,NaN,NaT,021.0-0013-0500.0,None,1.0,NaT,0.500000,2025-10-07,1.0,2025-10-07 21:27:14
1,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,NaN,NaT,319-021,None,1.0,NaT,0.500000,2025-10-07,1.0,2025-10-07 21:27:14
2,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,NaN,NaT,Z-05-0602,None,1.0,NaT,0.500000,2025-10-07,1.0,2025-10-07 21:27:14
3,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,NaN,NaT,Z-05-0601,None,2.0,NaT,0.500000,2025-10-07,1.0,2025-10-07 21:27:14
4,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,NaN,NaT,None,None,1.0,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9027,2025-10-07,970149281,"HUDSON, VICTORIA A.",Equity Line of Credit,25000.0,VAR,MTG,MG60,ACT,0.0725,...,NaN,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:03
9028,2025-10-07,970158171,"NOISEUX, KENNETH E.",Special HELOC,100000.0,VAR,MTG,MG52,ACT,0.0725,...,NaN,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:03
9029,2025-10-07,970170501,"MOURA, JOAO B.",Special HELOC,20000.0,VAR,MTG,MG52,ACT,0.0725,...,NaN,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:03
9030,2025-10-07,970186001,"SHURTLEFF, GIDEON N.",Special HELOC,38925.0,VAR,MTG,MG52,ACT,0.0725,...,NaN,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:05


In [33]:
heloc_non_flood_to_exclude = ['MG52','MG55','MG48','MG71']
mask_to_exclude = (
    (merged_df['currmiaccttypcd'].isin(heloc_non_flood_to_exclude)) &
    (merged_df['floodzoneyn'] == 'N')
)

In [34]:
# Filter out HELOCs that are not in flood zones
merged_df = merged_df[~mask_to_exclude].copy()

In [35]:
merged_df

,effdate_x,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,propyearnbr,occupancyeffdate,parcelnbr,fixedmarginamt,priorityseqnbr,datereleased,calcmarginpct,rundate,nbrofunits,datelastmaint_y
0,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,NaN,NaT,021.0-0013-0500.0,None,1.0,NaT,0.500000,2025-10-07,1.0,2025-10-07 21:27:14
1,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,NaN,NaT,319-021,None,1.0,NaT,0.500000,2025-10-07,1.0,2025-10-07 21:27:14
2,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,NaN,NaT,Z-05-0602,None,1.0,NaT,0.500000,2025-10-07,1.0,2025-10-07 21:27:14
3,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,NaN,NaT,Z-05-0601,None,2.0,NaT,0.500000,2025-10-07,1.0,2025-10-07 21:27:14
4,2025-10-07,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.0635,...,NaN,NaT,None,None,1.0,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9023,2025-10-07,970110341,"JENNESS, RANDALL",Equity LOC,0.0,VAR,MTG,MG50,ACT,0.0725,...,NaN,NaT,00034 0002A,None,1.0,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:03
9024,2025-10-07,970131221,"PETER, DAVID M.",Equity Line of Credit,250000.0,VAR,MTG,MG60,ACT,0.0725,...,NaN,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:03
9025,2025-10-07,970137541,"WILBER, MARILYN A.",Equity Line of Credit,20000.0,VAR,MTG,MG60,ACT,0.0725,...,NaN,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:03
9026,2025-10-07,970141021,"ARNOLD, NORMAN S.",Equity Line of Credit,50000.0,VAR,MTG,MG60,ACT,0.0725,...,NaN,NaT,None,None,NaN,NaT,1.000000,2025-10-07,NaN,2025-10-07 21:27:03


In [36]:
TABLE_PATH = src.config.SILVER / "insurance"
insurance = DeltaTable(TABLE_PATH).to_pandas()

In [37]:
insurance

,intrpolicynbr,effdate_link,inactivedate_link,datelastmaint_link,escrowyn_link,lenderfundedyn_link,escdisbchangeeffdate_link,load_timestamp_utc_link,rundate,insorgnbr,...,instypdesc,datelastmaint_policy,escrowyn_policy,lenderfundedyn_policy,escdisbchangeeffdate_policy,load_timestamp_utc_policy,load_timestamp_utc,escrowyn,lenderfundedyn,escdisbchangeeffdate
0,4213,2019-05-01,NaT,2025-04-28 07:46:30,None,None,NaT,2025-10-08 13:46:07.263638+00:00,2025-10-07,1106,...,Homeowner's Insurance,2025-10-07 21:27:17,None,None,None,2025-10-08 13:46:07.263638+00:00,2025-10-08 13:58:53.290530+00:00,Y,N,2019-05-01
1,5446,2020-02-05,NaT,2020-07-26 23:28:41,None,None,NaT,2025-10-08 13:46:07.263638+00:00,2025-10-07,1128,...,Homeowner's Insurance,2025-10-07 21:27:17,None,None,None,2025-10-08 13:46:07.263638+00:00,2025-10-08 13:58:53.290530+00:00,Y,N,2021-02-01
2,5694,2020-03-11,NaT,2020-07-26 23:28:41,None,None,NaT,2025-10-08 13:46:07.263638+00:00,2025-10-07,1134,...,Homeowner's Insurance,2025-10-07 21:27:17,None,None,None,2025-10-08 13:46:07.263638+00:00,2025-10-08 13:58:53.290530+00:00,Y,N,2020-03-01
3,5697,2020-05-01,NaT,2023-02-13 10:53:20,None,None,NaT,2025-10-08 13:46:07.263638+00:00,2025-10-07,1204,...,Private Mortgage Insurance,2025-10-07 21:27:17,None,None,None,2025-10-08 13:46:07.263638+00:00,2025-10-08 13:58:53.290530+00:00,Y,N,2020-05-01
4,5698,2020-03-12,NaT,2024-12-17 14:41:00,None,None,NaT,2025-10-08 13:46:07.263638+00:00,2025-10-07,1151,...,Homeowner's Insurance,2025-10-07 21:27:17,None,None,None,2025-10-08 13:46:07.263638+00:00,2025-10-08 13:58:53.290530+00:00,N,N,2021-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8686,10485,2022-05-02,NaT,2025-08-26 11:05:40,None,None,NaT,2025-10-08 13:46:07.263638+00:00,2025-10-07,1194,...,Homeowner's Insurance,2025-10-07 21:27:17,None,None,None,2025-10-08 13:46:07.263638+00:00,2025-10-08 13:58:53.290530+00:00,Y,N,2022-08-01
8687,11646,2022-07-29,NaT,2025-07-07 08:41:19,None,None,NaT,2025-10-08 13:46:07.263638+00:00,2025-10-07,1141,...,Homeowner's Insurance,2025-10-07 21:27:16,None,None,None,2025-10-08 13:46:07.263638+00:00,2025-10-08 13:58:53.290530+00:00,Y,N,2023-07-01
8688,11764,2022-08-15,NaT,2025-07-23 10:12:06,None,None,NaT,2025-10-08 13:46:07.263638+00:00,2025-10-07,1124,...,Homeowner's Insurance,2025-10-07 21:27:17,None,None,None,2025-10-08 13:46:07.263638+00:00,2025-10-08 13:58:53.290530+00:00,Y,N,2023-07-01
8689,11807,2022-08-17,NaT,2024-08-20 08:24:20,None,None,NaT,2025-10-08 13:46:07.263638+00:00,2025-10-07,1163,...,Homeowner's Insurance,2025-10-07 21:27:16,None,None,None,2025-10-08 13:46:07.263638+00:00,2025-10-08 13:58:53.290530+00:00,Y,N,2023-08-01


In [38]:
insurance['instypcd'].unique()

array(['HOME', 'PMI', 'FPLC', 'FLOD', 'BRSK', 'CNDO', 'COML', 'LIAB',
       'FORP', 'CTS'], dtype=object)

In [39]:
# Exclude PMI
insurance = insurance[insurance['instypcd'] != "PMI"].copy()

In [40]:
insurance.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 7877 entries, 0 to 8690
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   intrpolicynbr                7877 non-null   object             
 1   effdate_link                 7877 non-null   datetime64[us]     
 2   inactivedate_link            2 non-null      datetime64[us]     
 3   datelastmaint_link           7877 non-null   datetime64[us]     
 4   escrowyn_link                0 non-null      object             
 5   lenderfundedyn_link          0 non-null      object             
 6   escdisbchangeeffdate_link    0 non-null      datetime64[us]     
 7   load_timestamp_utc_link      7877 non-null   datetime64[us, UTC]
 8   rundate                      7877 non-null   datetime64[us]     
 9   insorgnbr                    7877 non-null   int64              
 10  instypcd                     7877 non-null   object  

In [41]:
insurance = insurance[[
    'intrpolicynbr',
    'instypcd',
    'instypdesc',
    'insorgnbr',
    'escrowyn',
    'premamt',
    'effdate_policy',
    'expiredate',
    'inactivedate_policy',
    'coverageamt'
]].copy()

In [42]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7877 entries, 0 to 8690
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   intrpolicynbr        7877 non-null   object        
 1   instypcd             7877 non-null   object        
 2   instypdesc           7877 non-null   object        
 3   insorgnbr            7877 non-null   int64         
 4   escrowyn             7877 non-null   object        
 5   premamt              7830 non-null   object        
 6   effdate_policy       7877 non-null   datetime64[us]
 7   expiredate           7393 non-null   datetime64[us]
 8   inactivedate_policy  1660 non-null   datetime64[us]
 9   coverageamt          7413 non-null   object        
dtypes: datetime64[us](3), int64(1), object(6)
memory usage: 676.9+ KB


In [43]:
insurance['insorgnbr'] = insurance['insorgnbr'].astype(str)

In [44]:
TABLE_PATH = src.config.BRONZE / "wh_org"
wh_org = DeltaTable(TABLE_PATH).to_pandas()

In [45]:
wh_org['orgnbr'] = wh_org['orgnbr'].astype(str)

In [46]:
wh_org = wh_org[['orgnbr','orgname']].copy()

In [47]:
wh_org = wh_org.rename(columns={'orgname':'Insurance Company Name'}).copy()

In [48]:
insurance = insurance.merge(wh_org, how='left', left_on='insorgnbr', right_on='orgnbr')

In [49]:
insurance

,intrpolicynbr,instypcd,instypdesc,insorgnbr,escrowyn,premamt,effdate_policy,expiredate,inactivedate_policy,coverageamt,orgnbr,Insurance Company Name
0,4213,HOME,Homeowner's Insurance,1106,Y,2935.00,2019-05-01,2026-05-22,NaT,590000.00,1106,ARBELLA INSURANCE GROUP
1,5446,HOME,Homeowner's Insurance,1128,Y,3150.00,2020-02-05,2021-02-01,NaT,350000.00,1128,TRAVELERS INSURANCE
2,5694,HOME,Homeowner's Insurance,1134,Y,1742.00,2020-03-11,NaT,NaT,None,1134,UNITED PROPERTY INSURANCE
3,5698,HOME,Homeowner's Insurance,1151,N,1725.00,2020-03-12,2021-02-01,2021-03-10,587000.00,1151,BAY STATE INSURANCE
4,5960,HOME,Homeowner's Insurance,1012666,Y,None,2020-06-11,2020-10-01,NaT,160000.00,1012666,LEZAOLA THOMPSON INSURANCE
...,...,...,...,...,...,...,...,...,...,...,...,...
7872,10485,HOME,Homeowner's Insurance,1194,Y,3277.20,2022-05-02,2026-09-15,NaT,528745.00,1194,JOHNSON & JOHNSON INSURANCE
7873,11646,HOME,Homeowner's Insurance,1141,Y,1182.00,2022-07-29,2026-07-28,NaT,653000.00,1141,HOMESITE INSURANCE
7874,11764,HOME,Homeowner's Insurance,1124,Y,1188.00,2022-08-15,2026-08-04,NaT,500000.00,1124,UNION MUTUAL
7875,11807,HOME,Homeowner's Insurance,1163,Y,4560.00,2022-08-17,2025-09-10,NaT,1199800.00,1163,SAFECO INSURANCE


In [50]:
insurance = insurance.drop(columns=['insorgnbr','orgnbr']).copy()

In [51]:
TABLE_PATH = src.config.SILVER / "acct_prop_ins_link"
acct_prop_ins_link = DeltaTable(TABLE_PATH).to_pandas()

In [52]:
acct_prop_ins_link

,acctnbr,propnbr,intrpolicynbr,load_timestamp_utc
0,201342911,30221,4213,2025-10-08 13:58:53.290530+00:00
1,150426875,50221,5446,2025-10-08 13:58:53.290530+00:00
2,201754331,32082,5694,2025-10-08 13:58:53.290530+00:00
3,150439381,50885,5697,2025-10-08 13:58:53.290530+00:00
4,150439381,50885,5698,2025-10-08 13:58:53.290530+00:00
...,...,...,...,...
8696,150741380,71134,10485,2025-10-08 13:58:53.290530+00:00
8697,150773896,73616,11646,2025-10-08 13:58:53.290530+00:00
8698,150243584,40256,11764,2025-10-08 13:58:53.290530+00:00
8699,201253881,29920,11807,2025-10-08 13:58:53.290530+00:00


In [53]:
acct_prop_ins_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8701 entries, 0 to 8700
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   acctnbr             8701 non-null   object             
 1   propnbr             8701 non-null   object             
 2   intrpolicynbr       8701 non-null   object             
 3   load_timestamp_utc  8701 non-null   datetime64[us, UTC]
dtypes: datetime64[us, UTC](1), object(3)
memory usage: 272.0+ KB


In [54]:
acct_prop_ins_link = acct_prop_ins_link[['propnbr','intrpolicynbr']].copy()
acct_prop_ins_link = acct_prop_ins_link.drop_duplicates().reset_index(drop=True)

In [55]:
acct_prop_ins_link

,propnbr,intrpolicynbr
0,30221,4213
1,50221,5446
2,32082,5694
3,50885,5697
4,50885,5698
...,...,...
8693,71134,10485
8694,73616,11646
8695,40256,11764
8696,29920,11807


In [56]:
merged_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 8063 entries, 0 to 9027
Data columns (total 141 columns):
 #    Column                  Dtype              
---   ------                  -----              
 0    effdate_x               datetime64[us]     
 1    acctnbr                 object             
 2    ownersortname           object             
 3    product                 object             
 4    noteopenamt             float64            
 5    ratetypcd               object             
 6    mjaccttypcd             object             
 7    currmiaccttypcd         object             
 8    curracctstatcd          object             
 9    noteintrate             float64            
 10   bookbalance             float64            
 11   notebal                 float64            
 12   contractdate            datetime64[us]     
 13   datemat                 datetime64[us]     
 14   taxrptfororgnbr         float64            
 15   taxrptforpersnbr        float64          

In [57]:
merged_df = merged_df[[
    'acctnbr',
    'ownersortname',
    'notebal',
    'Net Balance',
    'noteopenamt',
    'orig_ttl_loan_amt',
    'Full_Street_Address',
    'cityname',
    'statecd',
    'zipcd',
    'propnbr',
    'proptypcd',
    'proptypdesc',
    'propaddr1',
    'propaddr2',
    'propaddr3',
    'propcity',
    'propstate',
    'propzip',
    'propdesc',
    'floodzoneyn',
    'floodzone',
    'aprsvalueamt',
    'product',
    'mjaccttypcd',
    'currmiaccttypcd',
    'creditlimitamt',
    'contractdate'
]].copy()

In [58]:
merged_df = merged_df.rename(columns={
    'Full_Street_Address':'Borrower Primary Address',
    'cityname':'Borrower City',
    'statecd':'Borrower State',
    'zipcd': 'Borrower Zip'
}
).copy()

In [59]:
new_merged_df = merged_df.merge(acct_prop_ins_link, how='left',on='propnbr').merge(insurance, how='left', on='intrpolicynbr')

In [60]:
new_merged_df

,acctnbr,ownersortname,notebal,Net Balance,noteopenamt,orig_ttl_loan_amt,Borrower Primary Address,Borrower City,Borrower State,Borrower Zip,...,intrpolicynbr,instypcd,instypdesc,escrowyn,premamt,effdate_policy,expiredate,inactivedate_policy,coverageamt,Insurance Company Name
0,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
1,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
2,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
3,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
4,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9845,970110341,"JENNESS, RANDALL",0.0,0.0,0.0,28808.67,161 COUNTY ST,REHOBOTH,MA,02769,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
9846,970131221,"PETER, DAVID M.",0.0,0.0,250000.0,250000.00,18 SHORELINE DRIVE,FOXBOROUGH,MA,02035,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
9847,970137541,"WILBER, MARILYN A.",0.0,0.0,20000.0,20000.00,355 MT HOPE ST,NORTH ATTLEBORO,MA,02760,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
9848,970141021,"ARNOLD, NORMAN S.",0.0,0.0,50000.0,50000.00,1158 CEDARWOOD CIRCLE,NORTH DIGHTON,MA,02764,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN


In [61]:
# Property type grouping configuration
PROPERTY_TYPE_GROUPS = {
    'Autobody/Gas Station': ['Autobody/Gas Station','Gas Station and Convenience St','Auto-Truck Repair','Car Wash'],
    'Retail': ['Retail - Big Box Store','Shopping Plaza','Strip Plaza','General Retail','Dealership'],
    'Hospitality': ['Hotel/Motel','Hospitality/Event Space','Assisted Living'],
    'Recreation': ['Outdoor Recreation','Indoor Recreational','Golf Course','Marina'],
    'Industrial': ['Manufacturing','Warehouse','Industrial','Seafood Processing Plant','Solar Farm'],
    'Land': ['Land - Unimproved','Land - Improved','Parking Lot'],
    'Mixed Use': ['Mixed Use (Retail/Office)','Mixed Use (Retail/Residential)','Mixed Use (Office/Residential)'],
    'Multi Family': ['Apartment Building','Multi Family'],
    'General Office': ['Office - Professional','Office- General'],
    'Medical Office': ['Office - Medical'],
    'Restaurant': ['Restaurant'],
    'Residential': ['1-4 Fam Res - Non Own Occ','1 Family Residential - Own Occ','2 Family Residential - Own Occ','Condominium'],
    'Storage': ['Self Storage'],
    'Educational': ['Educational Facilities','Day Care'],
    'Religious': ['Church'],
    'Vehicles': ['Vehicle - Business','Boat'],
    'Other': ['Commercial - Other','Real Estate - Business','Real Estate - Bus&Bus Assets','Real Estate - Personal & Bus','Real Estate - Pers&Bus Assets','All Business Assets','Bus Assets w/Accts Receivable','UCC - ABA','UCC- Equipment','Assignment of Leases/Rents','General Contractor','Outdoor Dealers','Marketable Securities','SBA Loan','Funeral Home','Savings - Partially Secured','Passbook/Savings Secured']
}



In [62]:
# Create mapping from individual property types to groups
proptype_mapping = {code: group for group, codes in PROPERTY_TYPE_GROUPS.items() for code in codes}

# Add cleaned property type to property data
new_merged_df['Cleaned Prop Type'] = new_merged_df['proptypdesc'].map(proptype_mapping).fillna('Other')

In [63]:
new_merged_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9850 entries, 0 to 9849
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   acctnbr                   9850 non-null   object        
 1   ownersortname             9850 non-null   object        
 2   notebal                   9850 non-null   float64       
 3   Net Balance               9850 non-null   float64       
 4   noteopenamt               9850 non-null   float64       
 5   orig_ttl_loan_amt         9850 non-null   float64       
 6   Borrower Primary Address  9828 non-null   object        
 7   Borrower City             9828 non-null   object        
 8   Borrower State            9828 non-null   object        
 9   Borrower Zip              9828 non-null   object        
 10  propnbr                   9850 non-null   object        
 11  proptypcd                 9850 non-null   object        
 12  proptypdesc         

In [64]:
new_merged_df

,acctnbr,ownersortname,notebal,Net Balance,noteopenamt,orig_ttl_loan_amt,Borrower Primary Address,Borrower City,Borrower State,Borrower Zip,...,instypcd,instypdesc,escrowyn,premamt,effdate_policy,expiredate,inactivedate_policy,coverageamt,Insurance Company Name,Cleaned Prop Type
0,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Industrial
1,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Industrial
2,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Industrial
3,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Industrial
4,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9845,970110341,"JENNESS, RANDALL",0.0,0.0,0.0,28808.67,161 COUNTY ST,REHOBOTH,MA,02769,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Residential
9846,970131221,"PETER, DAVID M.",0.0,0.0,250000.0,250000.00,18 SHORELINE DRIVE,FOXBOROUGH,MA,02035,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Residential
9847,970137541,"WILBER, MARILYN A.",0.0,0.0,20000.0,20000.00,355 MT HOPE ST,NORTH ATTLEBORO,MA,02760,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Residential
9848,970141021,"ARNOLD, NORMAN S.",0.0,0.0,50000.0,50000.00,1158 CEDARWOOD CIRCLE,NORTH DIGHTON,MA,02764,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Residential


In [65]:
acct_role_link = DeltaTable(src.config.SILVER / "acct_role_link").to_pandas()

In [66]:
acct_role_link = acct_role_link[acct_role_link['acctrolecd'] == 'OWN']
acct_role_link = acct_role_link[[
    'acctnbr',
    'customer_id'
]].copy()
acct_role_link = acct_role_link.drop_duplicates(subset=['acctnbr'], keep='first')

In [67]:
base_cust_dim = DeltaTable(src.config.SILVER / "base_customer_dim").to_pandas()

In [68]:
base_cust_dim

,customer_id,customer_type,customer_name,adddate,taxidtypcd,taxid,distinct_loans,loan_net_balance,distinct_deposits,deposit_balance,distinct_other_products,other_balance,Active Account Owner,effdate,load_timestamp_utc
0,O1010530,Organization,LAWRENCE B COLALUCA,2004-10-21,FEIN,038441447,NaN,NaN,NaN,NaN,NaN,NaN,N,2025-10-07,2025-10-08 14:02:41.779481+00:00
1,O1006973,Organization,TAUNTON HOUSING AUTHORITY,2018-08-07,None,None,NaN,NaN,NaN,NaN,NaN,NaN,N,2025-10-07,2025-10-08 14:02:41.779481+00:00
2,O1007010,Organization,KATHLEEN PAIVA IRRV TR,2018-08-15,FEIN,467301937,NaN,NaN,NaN,NaN,NaN,NaN,N,2025-10-07,2025-10-08 14:02:41.779481+00:00
3,O1007083,Organization,AIR RITE HVAC INC,2018-09-06,FEIN,464243024,NaN,NaN,NaN,NaN,NaN,NaN,N,2025-10-07,2025-10-08 14:02:41.779481+00:00
4,O1007119,Organization,HENRY T MUNROE TRUST,2018-09-19,FEIN,836318503,NaN,NaN,NaN,NaN,NaN,NaN,N,2025-10-07,2025-10-08 14:02:41.779481+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186533,P1172611,Person,SOILEN MAZUMDER,2025-04-09,None,024880669,NaN,NaN,NaN,NaN,NaN,NaN,N,2025-10-07,2025-10-08 14:02:41.779481+00:00
186534,P1172657,Person,DEBORAH REYNOLDS,2025-04-10,None,309743324,1.0,653411.04,0.0,0.0,0.0,0.0,Y,2025-10-07,2025-10-08 14:02:41.779481+00:00
186535,P1172658,Person,JEFFREY LYONS,2025-04-10,None,033720581,NaN,NaN,NaN,NaN,NaN,NaN,N,2025-10-07,2025-10-08 14:02:41.779481+00:00
186536,P1172833,Person,BRENDON BOTELHO,2025-04-16,None,029765403,1.0,49288.63,0.0,0.0,0.0,0.0,Y,2025-10-07,2025-10-08 14:02:41.779481+00:00


In [69]:
base_cust_dim = base_cust_dim[[
    'customer_id',
    'customer_name'
]].copy()

In [70]:
base_cust_dim = base_cust_dim.rename(columns={
    'customer_name':'Primary Borrower 2'
}).copy()
acct_role_link = acct_role_link.merge(base_cust_dim, on='customer_id', how='left')

In [71]:
acct_role_link

,acctnbr,customer_id,Primary Borrower 2
0,101202,P1121200,MICHAEL F. BARTOMIOLI JR
1,101207,P1121986,NANCY C. FOGARTY
2,101209,P1120725,HENRY W. HATCHER
3,101211,P1120664,STEVEN H. HATCHER
4,101259,P1122462,FRANCIS M. DICARLI
...,...,...,...
49735,151220010,O1018740,SHARED PROSPERITY LAW PLLC
49736,151223155,O1018796,LAW OFFICE OF ANDREW J BEECHINOR
49737,151226737,O1018881,THE TURNER FAMILY TRUST
49738,151229236,O1012141,721 BELMONT ST LLC


In [72]:
acct_role_link = acct_role_link[[
    'acctnbr',
    'Primary Borrower 2'
]].copy()

In [73]:
new_merged_df = new_merged_df.merge(acct_role_link, on='acctnbr', how='left')

In [74]:
new_merged_df

,acctnbr,ownersortname,notebal,Net Balance,noteopenamt,orig_ttl_loan_amt,Borrower Primary Address,Borrower City,Borrower State,Borrower Zip,...,instypdesc,escrowyn,premamt,effdate_policy,expiredate,inactivedate_policy,coverageamt,Insurance Company Name,Cleaned Prop Type,Primary Borrower 2
0,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Industrial,NBPIV CONSTITUTION LLC
1,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Industrial,NBPIV CONSTITUTION LLC
2,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Industrial,NBPIV CONSTITUTION LLC
3,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Industrial,NBPIV CONSTITUTION LLC
4,151058057,NBPIV SARATOGA LLC,26000000.0,26000000.0,26000000.0,26000000.00,401 EDGEWATER PLACE SUITE 265,WAKEFIELD,MA,01880,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Other,NBPIV CONSTITUTION LLC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9845,970110341,"JENNESS, RANDALL",0.0,0.0,0.0,28808.67,161 COUNTY ST,REHOBOTH,MA,02769,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Residential,BRENDA L. JENNESS
9846,970131221,"PETER, DAVID M.",0.0,0.0,250000.0,250000.00,18 SHORELINE DRIVE,FOXBOROUGH,MA,02035,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Residential,JOY L. PETER
9847,970137541,"WILBER, MARILYN A.",0.0,0.0,20000.0,20000.00,355 MT HOPE ST,NORTH ATTLEBORO,MA,02760,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Residential,NaN
9848,970141021,"ARNOLD, NORMAN S.",0.0,0.0,50000.0,50000.00,1158 CEDARWOOD CIRCLE,NORTH DIGHTON,MA,02764,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,Residential,CAROL B. ARNOLD


In [75]:
new_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9850 entries, 0 to 9849
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   acctnbr                   9850 non-null   object        
 1   ownersortname             9850 non-null   object        
 2   notebal                   9850 non-null   float64       
 3   Net Balance               9850 non-null   float64       
 4   noteopenamt               9850 non-null   float64       
 5   orig_ttl_loan_amt         9850 non-null   float64       
 6   Borrower Primary Address  9828 non-null   object        
 7   Borrower City             9828 non-null   object        
 8   Borrower State            9828 non-null   object        
 9   Borrower Zip              9828 non-null   object        
 10  propnbr                   9850 non-null   object        
 11  proptypcd                 9850 non-null   object        
 12  proptypdesc         

In [76]:
new_merged_df = new_merged_df.rename(columns={
    'acctnbr':'Loan_Number',
    'ownersortname':'Borrower_Name',
    'Primary Borrower 2':'Borrower_Name_2',
    'Borrower Primary Address':'Borrower_Street_1',
    # Null row for Borrower Street 2
    'Borrower City':'Borrower_City',
    'Borrower State':'Borrower_State',
    'Borrower Zip':'Borrower_Zip',
    'notebal':'Principal_Balance',
    'creditlimitamt':'Line_Of_Credit_Amount',
    'contractdate':'Origination_Date',
    # Calculated Field for Collateral_Type = 'Real Estate'
    'propaddr1':'Property_Street_1',
    'propaddr2':'Property_Street_2',
    'propcity':'Property_City',
    'propstate':'Property_State',
    'propzip':'Property_Zip',
    'coverageamt':'Insurable_Value',
    # Replacement_Cost null field
    'floodzone':'Flood_Zone', # SWAP to actual flood zone description
    'instypcd':'Coverage_Type',
    'escrowyn':'Premium_Escrowed',
    'mjaccttypcd':'Loan_Type',
    'proptypdesc':'Building_Type',
    # ---
    'propdesc':'Collateral_Description'
}).copy()

new_merged_df['Collateral_Type'] =  'Real Estate'
new_merged_df['Borrower_Street_2'] = None
new_merged_df['Replacement_Cost'] = None

cleaned_df = new_merged_df[[
'Loan_Number',
'Borrower_Name',
'Borrower_Name_2',
'Borrower_Street_1',
'Borrower_Street_2',
'Borrower_City',
'Borrower_State',
'Borrower_Zip',
'Principal_Balance',
'Line_Of_Credit_Amount',
'Origination_Date',
'Collateral_Type',
'Property_Street_1',
'Property_Street_2',
'Property_City',
'Property_State',
'Property_Zip',
'Insurable_Value',
'Replacement_Cost',
'Flood_Zone',
'Coverage_Type',
'Premium_Escrowed',
'Loan_Type',
'Building_Type',
# Additional fields not on requested template
'Collateral_Description',
'currmiaccttypcd',
'product'
]].copy()


In [77]:
cleaned_df

,Loan_Number,Borrower_Name,Borrower_Name_2,Borrower_Street_1,Borrower_Street_2,Borrower_City,Borrower_State,Borrower_Zip,Principal_Balance,Line_Of_Credit_Amount,...,Insurable_Value,Replacement_Cost,Flood_Zone,Coverage_Type,Premium_Escrowed,Loan_Type,Building_Type,Collateral_Description,currmiaccttypcd,product
0,151058057,NBPIV SARATOGA LLC,NBPIV CONSTITUTION LLC,401 EDGEWATER PLACE SUITE 265,None,WAKEFIELD,MA,01880,26000000.0,0.00,...,NaN,None,X,NaN,NaN,CML,Warehouse,"36 Saratoga Blvd, Devens, MA 01434",CM40,Commercial Mortgages
1,151058057,NBPIV SARATOGA LLC,NBPIV CONSTITUTION LLC,401 EDGEWATER PLACE SUITE 265,None,WAKEFIELD,MA,01880,26000000.0,0.00,...,NaN,None,X,NaN,NaN,CML,Warehouse,"140 Constitution Blvd, Franklin, MA 02038",CM40,Commercial Mortgages
2,151058057,NBPIV SARATOGA LLC,NBPIV CONSTITUTION LLC,401 EDGEWATER PLACE SUITE 265,None,WAKEFIELD,MA,01880,26000000.0,0.00,...,NaN,None,X,NaN,NaN,CML,Warehouse,"589 Commerce Drive, Fall River, MA 02720",CM40,Commercial Mortgages
3,151058057,NBPIV SARATOGA LLC,NBPIV CONSTITUTION LLC,401 EDGEWATER PLACE SUITE 265,None,WAKEFIELD,MA,01880,26000000.0,0.00,...,NaN,None,X,NaN,NaN,CML,Warehouse,"599 Commerce Drive, Fall River, MA 02720",CM40,Commercial Mortgages
4,151058057,NBPIV SARATOGA LLC,NBPIV CONSTITUTION LLC,401 EDGEWATER PLACE SUITE 265,None,WAKEFIELD,MA,01880,26000000.0,0.00,...,NaN,None,None,NaN,NaN,CML,UCC Filing / Assignment,"fixtures, machinery, equipment etc on property...",CM40,Commercial Mortgages
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9845,970110341,"JENNESS, RANDALL",BRENDA L. JENNESS,161 COUNTY ST,None,REHOBOTH,MA,02769,0.0,28808.67,...,NaN,None,X,NaN,NaN,MTG,1 Family Residential - Own Occ,"161 COUNTY ST REHOBOTH, MA 02769",MG50,Equity LOC
9846,970131221,"PETER, DAVID M.",JOY L. PETER,18 SHORELINE DRIVE,None,FOXBOROUGH,MA,02035,0.0,0.00,...,NaN,None,None,NaN,NaN,MTG,1 Family Residential - Own Occ,"81 Childs River Rd, Falmouth, MA 02536",MG60,Equity Line of Credit
9847,970137541,"WILBER, MARILYN A.",NaN,355 MT HOPE ST,None,NORTH ATTLEBORO,MA,02760,0.0,0.00,...,NaN,None,None,NaN,NaN,MTG,1 Family Residential - Own Occ,"355 MT HOPE ST, NORTH ATTLEBORO, MA 02760 COUN...",MG60,Equity Line of Credit
9848,970141021,"ARNOLD, NORMAN S.",CAROL B. ARNOLD,1158 CEDARWOOD CIRCLE,None,NORTH DIGHTON,MA,02764,0.0,0.00,...,NaN,None,None,NaN,NaN,MTG,1 Family Residential - Own Occ,"1158 CEDARWOOD CIR, N DIGHTON, MA 02764 COUNTY...",MG60,Equity Line of Credit


In [78]:


# List of terms to exclude (case insensitive)
exclude_terms = [
    "All Business Assets",
    "UCC",
    "Assign",
    "Land",
    "Vehicle",
    "Equipment",
    "Bus Assets w/Acct Receivable",
    "Bus Assets",
    "Boat",
    "Passbook",
    "Certificate of Deposit",
    "Cash Life Insurance",
    "Parking Lot",
    "Marketable Securities",
    "Accounts Receivable",
    "Savings",
    "Classic Auto",
    "Inventory",
    "Mobile Home",
    "Lease Hold Mortgage",
    "Key Person life insurance",
    "security agreement",
    "stock",
    "Vessel"
]

# Function to check if any exclude term is in the collateral type (case insensitive)
def should_exclude(collateral_type):
    ct_lower = collateral_type.lower()
    return any(term.lower() in ct_lower for term in exclude_terms)

# Apply the filter
cleaned_df = cleaned_df[~cleaned_df['Building_Type'].apply(should_exclude)]


In [79]:
# Get Participations purchased to exclude
wh_acctuserfields = DeltaTable(src.config.BRONZE / "wh_acctuserfields").to_pandas()
parp = wh_acctuserfields[wh_acctuserfields['acctuserfieldcd'] == 'PARP'].copy()
assert parp['acctnbr'].is_unique, "Dupes on participation purchased user field"
parp_schema = {
    'acctnbr':'str'
}
parp = cdutils.input_cleansing.cast_columns(parp, parp_schema)
parp = parp[[
    'acctnbr',
    'acctuserfieldvalue'
]].copy()
parp = parp.rename(columns={
    'acctuserfieldvalue':'Participation_Purchased'
})

cleaned_df = cleaned_df.merge(parp, left_on='Loan_Number', right_on='acctnbr', how='left')

In [82]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7010 entries, 0 to 7009
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Loan_Number              7010 non-null   object        
 1   Borrower_Name            7010 non-null   object        
 2   Borrower_Name_2          3188 non-null   object        
 3   Borrower_Street_1        6991 non-null   object        
 4   Borrower_Street_2        0 non-null      object        
 5   Borrower_City            6991 non-null   object        
 6   Borrower_State           6991 non-null   object        
 7   Borrower_Zip             6991 non-null   object        
 8   Principal_Balance        7010 non-null   float64       
 9   Line_Of_Credit_Amount    7010 non-null   float64       
 10  Origination_Date         7010 non-null   datetime64[us]
 11  Collateral_Type          7010 non-null   object        
 12  Property_Street_1        6768 non-

In [83]:
cleaned_df = cleaned_df[cleaned_df['Participation_Purchased'].isnull()].copy()

In [ ]:
cleaned_df

In [84]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6972 entries, 0 to 7009
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Loan_Number              6972 non-null   object        
 1   Borrower_Name            6972 non-null   object        
 2   Borrower_Name_2          3183 non-null   object        
 3   Borrower_Street_1        6960 non-null   object        
 4   Borrower_Street_2        0 non-null      object        
 5   Borrower_City            6960 non-null   object        
 6   Borrower_State           6960 non-null   object        
 7   Borrower_Zip             6960 non-null   object        
 8   Principal_Balance        6972 non-null   float64       
 9   Line_Of_Credit_Amount    6972 non-null   float64       
 10  Origination_Date         6972 non-null   datetime64[us]
 11  Collateral_Type          6972 non-null   object        
 12  Property_Street_1        6730 non-null 

In [ ]:
# Filter out HE records where flood zone = 'X'
heloc_minors = ['MG52','MG55','MG48','MG71']

# if record in cleaned df['currmiaccttypcd'].isin(heloc_minors) and cleaned_df['Flood_Zone'] contains X , or TBD, or B, or C (case insensitive)
# then drop the record

# Afterward: Next operation is to set anything in floodzone that contains contains X , or TBD, or B, or C (case insensitive) to None


In [ ]:
OUTPUT_DIR = Path('./output')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
filename = 'lee_mason_extract.xlsx'
OUTPUT_PATH = OUTPUT_DIR / filename
cleaned_df.to_excel(OUTPUT_PATH, index=False) 